<a href="https://colab.research.google.com/github/leonardoLavagna/Drug-repurposing/blob/main/BI_LeoL_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [23]:
!git clone https://github.com/dinaghiassian/DIAMOnD DiaBLE
!git clone https://github.com/dinaghiassian/DIAMOnD
%pip install markov_clustering #https://github.com/GuyAllard/markov_clustering

Cloning into 'DiaBLE'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.
Cloning into 'DIAMOnD'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import math
import json
import os

import markov_clustering as mc
import statistics
import pandas as pd
import networkx as nx
import ast
import numpy as np
import pprint
import seaborn as sns
import collections
import math
import random
from sklearn.model_selection import train_test_split
import json
import sys
from scipy.stats import hypergeom

## 1) Get the data

In [4]:
# read the human interactome data
# https://downloads.thebiogrid.org/File/BioGRID/Release-Archive/BIOGRID-4.4.204/BIOGRID-ORGANISM-4.4.204.tab3.zip

homo_sapiens_network = pd.read_csv("/content/BIOGRID-ORGANISM-Homo_sapiens-4.4.204.tab3.txt", sep = "\t")

Filter out the same organism A and B:

In [5]:
homo_sapiens_network = homo_sapiens_network[homo_sapiens_network['Organism Name Interactor A'] == homo_sapiens_network['Organism Name Interactor B']]

Filter out the same Gene Interactor A and B (self-loops):

In [6]:
homo_sapiens_network = homo_sapiens_network[homo_sapiens_network['Official Symbol Interactor A'] != homo_sapiens_network['Official Symbol Interactor B']]

Filter out only the physical interactions:

In [7]:
homo_sapiens_network = homo_sapiens_network[homo_sapiens_network['Experimental System Type'] == "physical"]

To avoid duplicating edges A-B and B-A, we'll filter them in few steps later... Now we focus on the curated disease-gene association

In [13]:
gene_dis_association = pd.read_csv('/content/curated_gene_disease_associations.tsv', sep = '\t')
gene_dis_association = gene_dis_association[gene_dis_association['diseaseName'] == 'Hypertensive disease'] 

In [14]:
gene_dis_association

,geneId,geneSymbol,diseaseId,diseaseName,score,NofPmids,NofSnps,source
479,10019,SH2B3,C0020538,Hypertensive disease,0.203231,0.0,0.0,HPO
850,10052,GJC1,C0020538,Hypertensive disease,0.200000,1.0,0.0,CTD_human
874,10058,ABCB6,C0020538,Hypertensive disease,0.200000,0.0,0.0,HPO
2452,10269,ZMPSTE24,C0020538,Hypertensive disease,0.200000,0.0,0.0,HPO
3619,10451,VAV3,C0020538,Hypertensive disease,0.200275,1.0,0.0,CTD_human
...,...,...,...,...,...,...,...,...
51573,3569,IL6,C0020538,Hypertensive disease,0.233723,1.0,0.0,CTD_human
51781,358,AQP1,C0020538,Hypertensive disease,0.200000,1.0,0.0,CTD_human
52361,3630,INS,C0020538,Hypertensive disease,0.284381,2.0,0.0,CTD_human
52460,3636,INPPL1,C0020538,Hypertensive disease,0.203231,1.0,0.0,CTD_human


Effective number of disease genes present in the homo sapiens network:

In [15]:
disease_genes_in_interactome = set.intersection(set(gene_dis_association['geneSymbol'].tolist()), set(homo_sapiens_network['Official Symbol Interactor A'].tolist()).union(homo_sapiens_network['Official Symbol Interactor B'].tolist()))

### Create human interactome

In [16]:
human_interactome = nx.Graph()

In [17]:
human_interactome.add_nodes_from(map(int, list(set(homo_sapiens_network["Entrez Gene Interactor A"].tolist() + homo_sapiens_network["Entrez Gene Interactor B"].tolist())))) # considering the Gene IDs
# human_interactome.add_nodes_from(list(set(homo_sapiens_network["Official Symbol Interactor A"].tolist() + homo_sapiens_network["Official Symbol Interactor B"].tolist())))

Now we filter the duplicates edges A-B, B-A

In [18]:
human_interactome.add_edges_from(list(zip(map(int, homo_sapiens_network["Entrez Gene Interactor A"]), map(int, homo_sapiens_network["Entrez Gene Interactor B"]))))
# human_interactome.add_edges_from(zip(homo_sapiens_network["Official Symbol Interactor A"], homo_sapiens_network["Official Symbol Interactor B"]))

### LLC

In [19]:
LCCs = [len(human_interactome) for human_interactome in sorted(nx.connected_components(human_interactome), key=len, reverse=True)]
print(f"The largest component {LCCs[0]}") 

The largest component 3289


In [20]:
print(LCCs)

[3289, 6, 6, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [21]:
# extract first LLC
human_interactome_LCC = human_interactome.subgraph(max(nx.connected_components(human_interactome), key=len))
print(len(human_interactome_LCC)) # number of nodes in the largest component
print(len(human_interactome_LCC.edges()))

3289
6390


In [22]:
# save graph
nx.write_adjlist(human_interactome_LCC, "test.adjlist")

### Compile seed gene list

In [32]:
curated_gene_desease_association_DF = pd.read_csv("/content/curated_gene_disease_associations.tsv", sep = '\t', compression = 'infer')

In [34]:
disease_DF = curated_gene_desease_association_DF.loc[curated_gene_desease_association_DF['diseaseId'] == 'C0020538']
print('Number of detected genes involved in the desease: ', len(disease_DF))
disease_DF.head(5)

Number of detected genes involved in the desease:  303


,geneId,geneSymbol,diseaseId,diseaseName,score,NofPmids,NofSnps,source
479,10019,SH2B3,C0020538,Hypertensive disease,0.203231,0,0,HPO
850,10052,GJC1,C0020538,Hypertensive disease,0.200000,1,0,CTD_human
874,10058,ABCB6,C0020538,Hypertensive disease,0.200000,0,0,HPO
2452,10269,ZMPSTE24,C0020538,Hypertensive disease,0.200000,0,0,HPO
3619,10451,VAV3,C0020538,Hypertensive disease,0.200275,1,0,CTD_human


In [35]:
seed_list = list(disease_DF['geneId'])
print("List of Entrez genes:\n", *seed_list)

List of Entrez genes:
 10019 10052 10058 10269 10451 1071 10806 10911 10928 11020 111 11255 1161 118 1187 1215 1277 1281 1285 1286 1287 1371 1395 1401 1490 150 153 1535 1536 1543 156 1584 1585 1586 1589 1636 1813 183 1847 185 186 1889 1906 1908 1909 1956 196 2006 2056 2072 2074 210 213 2160 2161 2200 221357 2243 2263 22897 2294 22954 2301 2328 23327 2335 2353 23607 246 2475 2538 2542 26 26119 26146 26160 26249 2641 2643 2697 2701 2702 27020 27031 27130 2717 27241 2729 27347 2740 2778 2784 2796 2811 2876 2908 2920 2932 2952 2953 2982 301 3091 3145 3162 3240 3290 3291 335 3357 338 3383 3425 3479 348 348932 3553 3569 358 3630 3636 367 3717 3725 3758 3762 3779 3816 3827 389036 3949 3952 3953 3972 4000 4015 4023 4057 4069 4088 4089 4158 4190 4306 4313 4318 4363 4511 4512 4513 4514 4519 4535 4540 4541 4556 4558 4566 4567 4572 4574 4575 4577 4578 4582 4615 4627 4629 4638 475 476 4763 477 4830 4843 4846 486 4867 4878 4879 4881 490 492 4973 4982 5020 50485 5054 5076 50940 51057 51083 5139 5167 

In [37]:
print("List of gene symbols:\n", *list(disease_DF['geneSymbol']))
seed_genes_symbols = list(disease_DF['geneSymbol'])

List of gene symbols:
 SH2B3 GJC1 ABCB6 ZMPSTE24 VAV3 CETP SDCCAG8 UTS2 RALBP1 IFT27 ADCY5 HRH3 ERCC8 ADD1 CLCNKA CMA1 COL1A1 COL3A1 COL4A3 COL4A4 COL4A5 CPOX CRHR2 CRP CTGF ADRA2A ADRB1 CYBA CYBB CYP1A1 GRK2 CYP11B1 CYP11B2 CYP17A1 CYP21A2 ACE DRD2 AGT DUSP5 AGTR1 AGTR2 ECE1 EDN1 EDN3 EDNRA EGFR AHR ELN EPO ERCC4 ERCC6 ALAD ALB F11 F12 FBN1 GSTA5 FGA FGFR2 CEP164 FOXF1 TRIM32 FOXE3 FMO3 NEDD4L FN1 FOS CD2AP ALOX15 MTOR G6PC SLC37A4 AOC1 LDLRAP1 TRAF3IP1 IFT172 KLHL3 GCG GCH1 GJA1 GJA4 GJA5 NPTN NPHP3 INVS GLA BBS9 GCLC STK39 GLP1R GNAS GNB3 GNRH1 GP1BA GPX1 NR3C1 CXCL2 GSK3B GSTT1 GSTT2 GUCY1A3 ANXA1 HIF1A HMBS HMOX1 HP HSD11B1 HSD11B2 APOA1 HTR2B APOB ICAM1 IDUA IGF1 APOE SLC6A18 IL1B IL6 AQP1 INS INPPL1 AR JAK2 JUN KCNJ1 KCNJ5 KCNMB1 KLK1 KNG1 ACT LDLR LEP LEPR LHB LMNA LOX LPL LTF LYZ SMAD3 SMAD4 MC2R MDH1 NR3C2 MMP2 MMP9 ABCC1 TRNC COX1 COX2 COX3 CYTB ND1 ND5 ND6 TRNE TRNF TRNK TRNL1 TRNQ TRNS1 TRNS2 TRNV TRNW MUC1 MYD88 MYH9 MYH11 MYLK ATOX1 ATP1A1 NF1 ATP1A2 NME1 NOS2 NOS3 FXYD2

Check if gene symbols for all genes in the seed gene list are updated and approved on the HGNC website (from https://www.genenames.org/tools/multi-symbol-checker/)

In [39]:
# Upload this csv in the multi-symbol checker tool of the HGNC: 
disease_DF['geneSymbol'].to_csv('gene_symbols.csv', index = False, header = False, line_terminator = ',')

In [42]:
disease_DF['geneId'].to_csv('geneId.csv', index = False, header = False, line_terminator = ',')

In [43]:
disease_DF

,geneId,geneSymbol,diseaseId,diseaseName,score,NofPmids,NofSnps,source
479,10019,SH2B3,C0020538,Hypertensive disease,0.203231,0,0,HPO
850,10052,GJC1,C0020538,Hypertensive disease,0.200000,1,0,CTD_human
874,10058,ABCB6,C0020538,Hypertensive disease,0.200000,0,0,HPO
2452,10269,ZMPSTE24,C0020538,Hypertensive disease,0.200000,0,0,HPO
3619,10451,VAV3,C0020538,Hypertensive disease,0.200275,1,0,CTD_human
...,...,...,...,...,...,...,...,...
126779,9445,ITM2B,C0020538,Hypertensive disease,0.200000,1,0,CTD_human
127097,948,CD36,C0020538,Hypertensive disease,0.209195,1,0,CTD_human
128361,9657,IQCB1,C0020538,Hypertensive disease,0.200000,0,0,HPO
130166,9935,MAFB,C0020538,Hypertensive disease,0.200000,0,0,HPO


In [ ]:
# Now for each protein in seed collect info on uniprot